<a href="https://colab.research.google.com/github/Pigwen/hands-on-sft/blob/main/CHAPTER_2_Loading_a_Quantized_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Reqs

模型参数量到内存占用量的(大致)转换公式
```
model_size_in_mb = (num_params * (bits_per_param / 8)) / 1e6
```

# Quantization in a Nutsshell

量化本质就是分箱，步骤和直方图类似：
* 定义FP32数字的可能范围
* 将其均匀的分成若干个箱子
* 对于每一个数，确定对应的区间(箱子)，并赋予其区间(箱子)id

In [2]:
import torch

torch.manual_seed(11)
weights = torch.randn(1000) * .07
weights.min(), weights.max()

(tensor(-0.2066), tensor(0.2097))

In [3]:
n_bins = 4
bins = torch.linspace(weights.min(), weights.max(), n_bins+1)
bin_width = bins[1] - bins[0]
bins, bin_width

(tensor([-0.2066, -0.1026,  0.0015,  0.1056,  0.2097]), tensor(0.1041))

In [4]:
bin_indexs = ((weights.view(-1, 1) > bins).to(torch.int).argmin(dim=-1) - 1)
print(weights[:20], bin_indexs[:20])

tensor([-0.0358,  0.0720, -0.0247,  0.0086, -0.0127, -0.1048,  0.0099, -0.0367,
        -0.0174, -0.0368,  0.2025, -0.0416,  0.0918,  0.0247, -0.0921, -0.0006,
         0.0174,  0.1101, -0.1148, -0.1115]) tensor([1, 2, 1, 2, 1, 0, 2, 1, 1, 1, 3, 1, 2, 2, 1, 1, 2, 3, 0, 0])


In [5]:
bin_values = bins[:-1]
first_bin = bin_values[0]
bin_values

tensor([-0.2066, -0.1026,  0.0015,  0.1056])

In [6]:
approx_values = bin_indexs * bin_width + first_bin
print(approx_values[:20])

tensor([-0.1026,  0.0015, -0.1026,  0.0015, -0.1026, -0.2066,  0.0015, -0.1026,
        -0.1026, -0.1026,  0.1056, -0.1026,  0.0015,  0.0015, -0.1026, -0.1026,
         0.0015,  0.1056, -0.2066, -0.2066])


In [7]:
print(weights[:20])

tensor([-0.0358,  0.0720, -0.0247,  0.0086, -0.0127, -0.1048,  0.0099, -0.0367,
        -0.0174, -0.0368,  0.2025, -0.0416,  0.0918,  0.0247, -0.0921, -0.0006,
         0.0174,  0.1101, -0.1148, -0.1115])


In [8]:
from torch import nn

mse_fn = nn.MSELoss()
mse_fn(approx_values, weights).sqrt()

tensor(0.0615)

In [9]:
def quantize(weights: torch.Tensor, n_bits: int = 8):
  assert n_bits <= 16

  n_bins = 2 ** n_bits
  bins = torch.linspace(weights.min(), weights.max(), n_bins+1)
  first_bin = bins[0]
  bin_width = bins[1] - first_bin
  bin_index = (weights.view(-1, 1) > bins).to(torch.int).argmin(dim=-1)
  return bin_index, bin_width, first_bin

def dequantize(bin_index: torch.Tensor, bin_width: float, first_bin: float):
  return bin_index * bin_width + first_bin


In [10]:
for i in [2, 4, 6, 8, 16]:
  res = quantize(weights, i)
  approx_values = dequantize(*res)
  print(f"{i} bits quantization")
  print(f"approx: {approx_values[:6]}")
  print(f"weights: {weights[:6]}")
  print(f"loss: {mse_fn(approx_values, weights).sqrt()}\n")

2 bits quantization
approx: tensor([ 0.0015,  0.1056,  0.0015,  0.1056,  0.0015, -0.1026])
weights: tensor([-0.0358,  0.0720, -0.0247,  0.0086, -0.0127, -0.1048])
loss: 0.059067390859127045

4 bits quantization
approx: tensor([-0.0245,  0.0796, -0.0245,  0.0275,  0.0015, -0.1026])
weights: tensor([-0.0358,  0.0720, -0.0247,  0.0086, -0.0127, -0.1048])
loss: 0.014644436538219452

6 bits quantization
approx: tensor([-0.0310,  0.0731, -0.0245,  0.0145, -0.0115, -0.1026])
weights: tensor([-0.0358,  0.0720, -0.0247,  0.0086, -0.0127, -0.1048])
loss: 0.003786345012485981

8 bits quantization
approx: tensor([-0.0343,  0.0731, -0.0245,  0.0096, -0.0115, -0.1042])
weights: tensor([-0.0358,  0.0720, -0.0247,  0.0086, -0.0127, -0.1048])
loss: 0.000910897389985621

16 bits quantization
approx: tensor([-0.0359,  0.0718, -0.0248,  0.0085, -0.0128, -0.1049])
weights: tensor([-0.0358,  0.0720, -0.0247,  0.0086, -0.0127, -0.1048])
loss: 0.00013679217954631895



# Half-Precision Weights

In [12]:
fp16_weights = weights.to(torch.float16)
print(fp16_weights[:6])
print(weights[:6])

tensor([-0.0358,  0.0720, -0.0247,  0.0086, -0.0127, -0.1048],
       dtype=torch.float16)
tensor([-0.0358,  0.0720, -0.0247,  0.0086, -0.0127, -0.1048])


In [13]:
print(mse_fn(fp16_weights, weights).sqrt())

tensor(1.4244e-05)


## Living on the Edge

In [15]:
torch.manual_seed(14)
tiny_values = torch.randn(1000) * 1e-5
fp16_tiny_values = tiny_values.to(torch.float16)
mse_fn(fp16_tiny_values, tiny_values)

tensor(2.8526e-16)

In [16]:
print(tiny_values[155:160])
print(fp16_tiny_values[155:160])

tensor([-2.7241e-06,  1.1441e-05,  3.7199e-06, -1.1252e-06, -2.4735e-08])
tensor([-2.7418e-06,  1.1444e-05,  3.6955e-06, -1.1325e-06, -0.0000e+00],
       dtype=torch.float16)


In [17]:
torch.manual_seed(19)
large_values = torch.randn(1000) * 1e5
fp16_large_values = large_values.to(torch.float16)
print(large_values[:5])
print(fp16_large_values[:5])

tensor([155074.0938,  64881.6602,   2729.5815, -40790.6562,  68846.7188])
tensor([    inf,  64896.,   2730., -40800.,     inf], dtype=torch.float16)


In [19]:
fp16_info = torch.finfo(torch.float16)
fp16_info

finfo(resolution=0.001, min=-65504, max=65504, eps=0.000976562, smallest_normal=6.10352e-05, tiny=6.10352e-05, dtype=float16)

In [20]:
bf16_info = torch.finfo(torch.bfloat16)
print(bf16_info)
print(fp16_info)

finfo(resolution=0.01, min=-3.38953e+38, max=3.38953e+38, eps=0.0078125, smallest_normal=1.17549e-38, tiny=1.17549e-38, dtype=bfloat16)
finfo(resolution=0.001, min=-65504, max=65504, eps=0.000976562, smallest_normal=6.10352e-05, tiny=6.10352e-05, dtype=float16)


In [21]:
fp32_info = torch.finfo(torch.float32)
print(fp32_info)
print(bf16_info)

finfo(resolution=1e-06, min=-3.40282e+38, max=3.40282e+38, eps=1.19209e-07, smallest_normal=1.17549e-38, tiny=1.17549e-38, dtype=float32)
finfo(resolution=0.01, min=-3.38953e+38, max=3.38953e+38, eps=0.0078125, smallest_normal=1.17549e-38, tiny=1.17549e-38, dtype=bfloat16)


# Loading Models

In [1]:
from collections import Counter
from transformers import AutoModelForCausalLM

def get_param_dtypes(iterable, top_k=3):
  return Counter([i.dtype for i in iterable]).most_common(top_k)

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m", device_map="auto")
print(model.get_memory_footprint()/1e6, get_param_dtypes(model.parameters()))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

1324.785664 [(torch.float32, 388)]


In [7]:
import torch
from transformers import AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m", device_map="auto", dtype=torch.float32)

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
batch = tokenizer(["This is a simple test"], return_tensors="pt")

batch["labels"] = batch["input_ids"] # 用于计算loss
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device is {device}")
batch = {k: v.to(device) for k, v in batch.items()}
output = model(**batch)
output.loss



device is cuda


tensor(3.8001, device='cuda:0', grad_fn=<NllLossBackward0>)

## Half-Precision Models(16-bit)

In [11]:
supported = torch.cuda.is_bf16_supported(including_emulation=False)
dtype16 = torch.bfloat16 if supported else torch.float16
dtype16

torch.float16

In [13]:
model.to(dtype16)
print(model.get_memory_footprint()/1e6, get_param_dtypes(model.parameters()))

662.392832 [(torch.float16, 388)]


# Mixed Precision